In [3]:
import cv2 as cv
import numpy as np

In [4]:

# --- Função para calcular o background médio ---
def avg_frame(video_path, max_frames=300, step=3):
    cap = cv.VideoCapture(video_path)
    count = 0
    avg = None
    frame_index = 0

    while count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_index % step != 0:
            frame_index += 1
            continue

        frame_index += 1
        frame_f = frame.astype(np.float32)

        if avg is None:
            avg = frame_f
        else:
            avg += frame_f

        count += 1

    cap.release()

    if count == 0:
        raise Exception(f"ERROR: No frames loaded from video: {video_path}")

    return (avg / count).astype(np.uint8)


In [5]:
# --- Função para detectar carros ---
def detect_cars(frame, background_blur, mask_roi):
    frame_blur = cv.GaussianBlur(frame, (21, 21), 0)
    diff = cv.absdiff(frame_blur, background_blur)
    gray = cv.cvtColor(diff, cv.COLOR_BGR2GRAY)
    masked = cv.bitwise_and(gray, mask_roi)
    _, thresh = cv.threshold(masked, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)

    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (7, 7))
    cleaned = cv.morphologyEx(thresh, cv.MORPH_CLOSE, kernel, iterations=2)
    cleaned = cv.morphologyEx(cleaned, cv.MORPH_OPEN, kernel, iterations=1)

    contours, _ = cv.findContours(cleaned, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    boxes = []

    for cnt in contours:
        area = cv.contourArea(cnt)
        if area < 800 or area > 10000:  # filtrar objetos muito pequenos ou muito grandes
            continue

        x, y, w, h = cv.boundingRect(cnt)
        ratio = w / float(h)
        if ratio < 0.3 or ratio > 3:  # filtrar objetos estreitos ou largos
            continue

        boxes.append((x, y, w, h))

    return boxes

In [6]:
# --- Função para calcular velocidade ---
def calculate_speed(prev_pos, curr_pos, fps, meters_per_pixel=0.25):
    px, py = prev_pos
    cx, cy = curr_pos
    distance_pixels = np.sqrt((cx - px) ** 2 + (cy - py) ** 2)
    distance_m = distance_pixels * meters_per_pixel
    speed_m_per_s = distance_m * fps
    speed_kmh = speed_m_per_s * 3.6
    return speed_kmh

In [7]:
# --- Função principal com tracking e velocidade ---
def count_cars_with_speed(video_path, output_path, roi_polygon, meters_per_pixel=0.25):
    background = avg_frame(video_path)
    background_blur = cv.GaussianBlur(background, (21, 21), 0)

    cap = cv.VideoCapture(video_path)
    w = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv.CAP_PROP_FPS)

    mask_roi = np.zeros((h, w), dtype=np.uint8)
    cv.fillPoly(mask_roi, [roi_polygon], 255)

    out = cv.VideoWriter(output_path, cv.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

    prev_centroids = {}
    next_id = 1
    car_speeds = {}
    car_age = {}
    car_speeds_history = {}

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        boxes = detect_cars(frame, background_blur, mask_roi)
        new_centroids = {}

        for (x, y, w2, h2) in boxes:
            cx = x + w2 // 2
            cy = y + h2 // 2

            assigned_id = None
            for obj_id, (px, py) in prev_centroids.items():
                if abs(cx - px) < 30 and abs(cy - py) < 40:
                    assigned_id = obj_id
                    break

            if assigned_id is None:
                assigned_id = next_id
                next_id += 1

            new_centroids[assigned_id] = (cx, cy)

            # Atualizar idade e histórico
            car_age[assigned_id] = car_age.get(assigned_id, 0) + 1
            if assigned_id not in car_speeds_history:
                car_speeds_history[assigned_id] = []

            # Calcular velocidade só se o carro existe há mais de 1 frame
            if assigned_id in prev_centroids and car_age[assigned_id] > 1:
                speed = calculate_speed(prev_centroids[assigned_id], (cx, cy), fps, meters_per_pixel)
            else:
                speed = 0.0

            car_speeds_history[assigned_id].append(speed)
            # Média móvel das últimas 3 velocidades
            speed_avg = np.mean(car_speeds_history[assigned_id][-3:])
            car_speeds[assigned_id] = speed_avg

            # Desenhar bounding box e velocidade
            cv.rectangle(frame, (x, y), (x+w2, y+h2), (0, 255, 0), 2)
            cv.putText(frame, f"ID:{assigned_id}", (x, y-20), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)
            cv.putText(frame, f"{speed_avg:.1f} km/h", (x, y-5), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255), 1)

        # Remover carros que desapareceram
        prev_centroids = new_centroids.copy()
        car_age = {cid: car_age[cid] for cid in prev_centroids.keys()}
        car_speeds_history = {cid: car_speeds_history[cid] for cid in prev_centroids.keys()}

        cv.polylines(frame, [roi_polygon], True, (255, 0, 0), 2)
        out.write(frame)
        cv.imshow("Car Counting & Speed", frame)

        if cv.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    out.release()
    cv.destroyAllWindows()

    print("Done! Cars counted:", next_id-1)
    for car_id, speed in car_speeds.items():
        print(f"Car {car_id}: {speed:.1f} km/h")




In [8]:
# --- USO ---
video_path = "resources/AutoEstrada.avi"

roi_polygon = np.array([
    [100, 70],
    [155, 70],
    [230, 240],
    [5, 240]
], dtype=np.int32)

count_cars_with_speed(video_path, "cars_counted_speed.mp4", roi_polygon)

Done! Cars counted: 12
Car 1: 97.7 km/h
Car 2: 106.2 km/h
Car 3: 97.9 km/h
Car 4: 102.1 km/h
Car 5: 99.4 km/h
Car 6: 101.0 km/h
Car 7: 72.8 km/h
Car 8: 175.2 km/h
Car 9: 79.1 km/h
Car 10: 202.7 km/h
Car 11: 0.0 km/h
Car 12: 148.3 km/h
